In [ ]:
import mxnet as mx
import numpy as np
import logging


logging.basicConfig(level=logging.INFO)


sample_count=1000
train_count=800
valid_count=sample_count-train_count

feature_count=100
category_count=10
batch=10

In [5]:
X=mx.nd.uniform(low=0,high=1,shape=(sample_count,feature_count))

In [6]:
X.shape

(1000L, 100L)

In [7]:
X.asnumpy()

array([[ 0.54881352,  0.59284461,  0.71518934, ...,  0.60783064,
         0.36371076,  0.32504722],
       [ 0.57019675,  0.03842543,  0.43860152, ...,  0.36980811,
         0.00469548,  0.14644176],
       [ 0.67781651,  0.5696184 ,  0.27000797, ...,  0.46924916,
         0.99884701,  0.84114015],
       ..., 
       [ 0.59305507,  0.41538411,  0.24317271, ...,  0.58364183,
         0.8284471 ,  0.4361586 ],
       [ 0.71750212,  0.5193429 ,  0.80764365, ...,  0.15779524,
         0.86435819,  0.72405148],
       [ 0.82241368,  0.72197711,  0.85116214, ...,  0.24351381,
         0.55131787,  0.12545156]], dtype=float32)

In [8]:
Y=mx.nd.empty((sample_count))
for i in range(0,sample_count-1):
    Y[i]=np.random.randint(0,category_count)

In [9]:
Y.shape

(1000L,)

In [44]:
Y[0:10].asnumpy()

AttributeError: 'DataBatch' object has no attribute 'asnumpy'

In [47]:
X_train=mx.nd.crop(X,begin=(0,0),end=(train_count,feature_count-1))
X_valid=mx.nd.crop(X,begin=(train_count,0),end=(sample_count,feature_count-1))
Y_train=Y[0:train_count]
Y_valid=Y[train_count:sample_count]

In [26]:
X_train.shape

(800L, 99L)

In [12]:
data=mx.sym.Variable('data')

In [13]:
fc1=mx.sym.FullyConnected(data,name='fc1',num_hidden=64)

In [14]:
relu1=mx.sym.Activation(fc1,name='relu1',act_type="relu")

In [15]:
fc2=mx.sym.FullyConnected(relu1,name='fc2',num_hidden=category_count)

In [16]:
out=mx.sym.SoftmaxOutput(fc2,name='softmax')
mod=mx.mod.Module(out)

In [48]:
train_iter=mx.io.NDArrayIter(data=X_train,label=Y_train,batch_size=batch)

In [19]:
train_iter.reset()

In [20]:
mod.bind(data_shapes=train_iter.provide_data,label_shapes=train_iter.provide_label)

In [21]:
mod.init_params(initializer=mx.init.Xavier(magnitude=2.))

In [22]:
mod.init_optimizer(optimizer='sgd',optimizer_params=(('learning_rate',0.1),))

In [23]:
mod.fit(train_iter,num_epoch=50)

/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.115000
INFO:root:Epoch[0] Time cost=0.134
INFO:root:Epoch[1] Train-accuracy=0.126250
INFO:root:Epoch[1] Time cost=0.102
INFO:root:Epoch[2] Train-accuracy=0.150000
INFO:root:Epoch[2] Time cost=0.090
INFO:root:Epoch[3] Train-accuracy=0.161250
INFO:root:Epoch[3] Time cost=0.089
INFO:root:Epoch[4] Train-accuracy=0.180000
INFO:root:Epoch[4] Time cost=0.075
INFO:root:Epoch[5] Train-accuracy=0.207500
INFO:root:Epoch[5] Time cost=0.102
INFO:root:Epoch[6] Train-accuracy=0.223750
INFO:root:Epoch[6] Time cost=0.089
INFO:root:Epoch[7] Train-accuracy=0.228750
INFO:root:Epoch[7] Time cost=0.104
INFO:root:Epoch[8] Train-accuracy=0.252500
INFO:root:Epoch[8] Time cost=0.110
INFO:root:Epoch[9] Train-accuracy=0.263750
INFO:root:Epoch[9] Time cost=0.098
IN

In [49]:
pred_iter=mx.io.NDArrayIter(data=X_valid,label=Y_valid,batch_size=batch)

In [50]:
pred_count=valid_count
correct_preds=total_correct_preds=0

for preds, i_batch, batch in mod.iter_predict(pred_iter):
    label=batch.label[0].asnumpy().astype(int)
    pred_label=preds[0].asnumpy().argmax(axis=1)
    correct_preds=np.sum(pred_label==label)
    total_correct_preds=total_correct_preds+correct_preds
    
print('Validation accuracy: %2.2f' %(1.0*total_correct_preds/pred_count))

Validation accuracy: 0.10
